In [1]:
#obtinerea scorurilor de atractivitate ca medie a notelor date 
import cv2
import numpy as np
from matplotlib import pyplot as plt
import math
import os
import glob

# path to the images and the text file which holds the scores and ids
base_images_path = r'C:\Users\user\Desktop\licenta\15000\AVA_dataset\imageslucru\\'
ava_dataset_path = r'C:\Users\user\Desktop\licenta\15000\AVA_dataset\AVAlucru.txt'

files = glob.glob(base_images_path + "*.jpg")
files = sorted(files)

train_image_paths = []
train_scores = []

pathToPhotos = "C:\\Users\\user\\Desktop\\licenta\\15000\\AVA_dataset\\imageslucru"
s=[]
for filename in os.listdir(pathToPhotos):
    getName = filename.split('.')
    s.append(int(getName[0]))
s.sort()
#print(s)

print("Loading training set and val set")
with open(ava_dataset_path, mode='r') as f:
    lines = f.readlines()
    for i, line in enumerate(lines):
        token = line.split()
        nume = int(token[1])
        values = np.array(token[2:12], dtype='float32')
        score = 0.0
        for i in range(len(values)):
            score += values[i]*(i+1)
        score /= values.sum()
        score = round(score,2)
        #print(score)

        file_path = base_images_path + str(nume) + '.jpg'
        if os.path.exists(file_path):
            train_image_paths.append(file_path)
            #train_scores.append(score)
        g = open("scores.txt", "a")
        g.write(str(score)+'\n')
        g.close()
         
train_image_paths = np.array(train_image_paths)
#train_scores = np.array(train_scores, dtype='float32')

val_image_paths = train_image_paths[-200:]
#val_scores = train_scores[-200:]
train_image_paths = train_image_paths[:-200]
#train_scores = train_scores[:-200]

print('Train set size : ', train_image_paths.shape)
print('Val set size : ', val_image_paths.shape)
print('Train and validation datasets ready !')


Loading training set and val set
Train set size :  (14800,)
Val set size :  (200,)
Train and validation datasets ready !


In [2]:
#obtinerea etichetelor pozelor folosite, din toate calculate
train_scores = []
with open('scores.txt', mode='r') as l:
    liness = l.readlines()
    for i, line in enumerate(liness):
        for j in range(0,15000):
            if(s[j]==i):
                k = open("scoresfinal.txt", "a")
                k.write(str(liness[i-1]))
                k.close()
                train_scores.append(liness[i-1])
                
    k = open("scoresfinal.txt", "a")
    k.write(str(liness[-1]))
    k.close()
    train_scores.append(liness[-1])
    
train_scores = np.array(train_scores, dtype='float32')
val_scores = train_scores[-200:]
train_scores = train_scores[:-200]

print('Train set size : ', train_image_paths.shape, train_scores.shape)
print('Val set size : ', val_image_paths.shape, val_scores.shape)
print('Train and validation datasets ready !')

Train set size :  (14800,) (14800,)
Val set size :  (200,) (200,)
Train and validation datasets ready !


In [3]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import math
import os
import glob
from past.builtins import xrange
import sys
#obtinerea descriptorilor tuturor celor 15000 de imagini
base_images_path = r'C:\Users\user\Desktop\licenta\15000\AVA_dataset\imageslucru\\'
ava_dataset_path = r'C:\Users\user\Desktop\licenta\15000\AVA_dataset\AVAlucru.txt'

files = glob.glob(base_images_path + "*.jpg")
files = sorted(files)

train_image_paths = []
with open(ava_dataset_path, mode='r') as f:
    lines = f.readlines()
    for i, line in enumerate(lines):
        token = line.split()
        id = int(token[0])
        
        file_path = base_images_path + str(id) + '.jpg'
        if os.path.exists(file_path):
            train_image_paths.append(file_path)
            
train_image_paths = np.array(train_image_paths)
print(train_image_paths.shape)



(15000,)


In [4]:
def color_descriptor(img):
    hsv_image = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    levels = 8
    hhist = cv2.calcHist([hsv_image],[0], None, [levels], [0, 180] )
    sumhhist = np.sum(hhist)
    hhist[:] = [x/sumhhist for x in hhist]
    shist = cv2.calcHist([hsv_image],[1], None, [levels], [0, 255] )
    sumshist = np.sum(shist)
    shist[:] = [x/sumshist for x in shist]
    vhist = cv2.calcHist([hsv_image],[2], None, [levels], [0, 255] )
    sumvhist = np.sum(vhist)
    vhist[:] = [x/sumvhist for x in vhist]
    colordescriptor = np.concatenate([hhist, shist, vhist]).flatten()
    return colordescriptor

def contrast_descriptor(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    layer=img.copy()
    gp=[layer]
    descriptor_contrast = 0
    for i in range(6):
        layer=cv2.pyrDown(layer)
        gp.append(layer)
    layer=gp[5]
    lap_pyr=[layer]
    for i in range(5,0,-1):
        size=(gp[i-1].shape[1],gp[i-1].shape[0])
        gaussian_extend=cv2.pyrUp(gp[i],dstsize=size)
        laplacian=cv2.subtract(gp[i-1],gaussian_extend)
        lap_pyr.append(laplacian)
    for i in range(1,6):
        m,n = lap_pyr[i].shape
        contrast_lvl = 0
        for j in range (m):
            for k in range (n):
                contrast_lvl  = contrast_lvl + int(lap_pyr[i][j][k])*int(lap_pyr[i][j][k])
        contrast_lvl = contrast_lvl/(m*n*np.power(2,5-i))
        descriptor_contrast = descriptor_contrast + contrast_lvl
    return(descriptor_contrast)

In [5]:
descriptori_img=np.zeros((15000,25))
#matricea trb sa aiba 15000 linii si 25 col
m=15000;
n=25;
for i in range(m):
    img=cv2.imread(train_image_paths[i])
    for j in range(24):
        descriptori_img[i][j]=(color_descriptor(img))[j]
    descriptori_img[i][n-1]=contrast_descriptor(img)
print(descriptori_img.shape)

(15000, 25)


In [6]:
for i in range(m):
    for j in range(n):
        k = open("image_descriptor.txt", "a")
        k.write(str(descriptori_img[i][j])+' ')
    k.write('\n')
    k.close()
print('Train descriptors ready!')

Train descriptors ready!
